### 读取json文件

In [ ]:
import pandas as pd

In [ ]:
interview = pd.read_json(r'../Data/文本划分/中国外交部例行记者会.json')

In [ ]:
interview

### 提取json文件内的内容并转为文本文件

In [ ]:
import json

with open(r'../Data/文本划分/中国外交部例行记者会.json', encoding='utf-8') as json_data:
    data = json.load(json_data)

In [ ]:
type(data)

In [ ]:
questions = []
answers = []
journals = []

for d in data:
    qa_list  = d.get('question_answer_list')
    for qa in qa_list:
        question = qa.get('question')
        answer = qa.get('answer')
        journal = qa.get('journal')
        questions.append(question)
        journals.append(journal)
        answers.append(answer)

In [ ]:
questions,journals, answers

In [ ]:
def list_txt(path, list=None):
    if list != None:
        file = open(path, 'w', encoding='utf-8')
        [file.write(str(item) + '\n') for item in list]
        file.close()
        return None
    else:
        file = open(path, 'r')
        rdlist = eval(file.read()) # 抛出异常
        file.close()
        return rdlist

In [ ]:
list_txt(r'C:\Users\hp\桌面\项目\数据挖掘\Data\文本划分\文本格式的问答文件\questions.txt', questions)
list_txt(r'C:\Users\hp\桌面\项目\数据挖掘\Data\文本划分\文本格式的问答文件\answers.txt', answers)

### 预览df格式的问答内容

In [ ]:
QaA = pd.DataFrame({'questions': questions, 'journals': journals, 'answers': answers})

In [ ]:
QaA

### 利用jieba进行分词

In [ ]:
import jieba

### 获取停用词

In [ ]:
def get_stopwords(paths):
    stopwords = []
    for path in paths:
        stopwords += [line.strip() for line in open(path, 'r', encoding='utf-8').readlines()]
    return stopwords

In [ ]:
stop_paths = [r'C:/Users/hp/桌面/stopwords-master/stopwords-master/cn_stopwords.txt',
            r'C:/Users/hp/桌面/stopwords-master/stopwords-master/hit_stopwords.txt',
            r'C:/Users/hp/桌面/stopwords-master/stopwords-master/scu_stopwords.txt']

### 分词并写入

In [ ]:
def seg_sentence(sentence, stop_paths):
    result = ""
    sentence_seged=jieba.cut(sentence.strip())
    stopwords = get_stopwords(stop_paths)
    for word in sentence_seged:
        if word not in stopwords:
            if word not in ['问', '答']:
                if word!='\t':
                    result += word
                    result +=" "
    return result

In [ ]:
questions = open(r'C:\Users\hp\桌面\项目\数据挖掘\Data\文本划分\文本格式的问答文件\questions.txt','r',
                                  encoding='utf-8')
outs = open(r"C:/Users/hp/桌面/项目/数据挖掘/Data/文本划分/分词后的文本/seg_result_Q.txt",'w',
                                  encoding='utf-8')

for line in questions:
    line_seg = seg_sentence(line, stop_paths)
    outs.write(line_seg + '\n')
    
outs.close()
questions.close()

In [ ]:
answers = open(r'C:/Users/hp/桌面/项目/数据挖掘/Data/文本划分/文本格式的问答文件/answers.txt', 'r',
                                  encoding='utf-8')
outs = open(r"C:/Users/hp/桌面/项目/数据挖掘/Data/文本划分/分词后的文本/seg_result_A.txt",'w',
                                  encoding='utf-8')

for line in answers:
    line_seg = seg_sentence(line, stop_paths)
    outs.write(line_seg + '\n')

outs.close()
answers.close()

### 获取分词以及词频，以字典形式存储

In [ ]:
def get_countdic(lists):
    count_dict = dict()
    for i in lists:
        if i in count_dict:
            count_dict[i] += 1
        else:
            count_dict[i] = 1
    return count_dict

In [ ]:
word_list = []
for line in open(r"C:/Users/hp/桌面/项目/数据挖掘/Data/文本划分/分词后的文本/seg_result_Q.txt", 'r',encoding='utf-8'):
        l = str(line).strip('\n').strip(" ")
        word_list += l.split(" ")
        dic = get_countdic(word_list)
with open(r"C:/Users/hp/桌面/项目/数据挖掘/Data/文本划分/分词后的文本/frequency_Q.txt", 'w',encoding='utf-8') as f:
    for k, v in dic.items():
        f.write('%s,%d\n' % (k, v))

In [ ]:
word_list = []
for line in open(r"C:/Users/hp/桌面/项目/数据挖掘/Data/文本划分/seg_result_A.txt", 'r',encoding='utf-8'):
        l = str(line).strip('\n').strip(" ")
        word_list += l.split(" ")
        dic = get_countdic(word_list)
with open(r"C:/Users/hp/桌面/项目/数据挖掘/Data/文本划分/frequency_A.txt", 'w',encoding='utf-8') as f:
    for k, v in dic.items():
        f.write('%s,%d\n' % (k, v))

### 读入词频和词语的txt文件

In [ ]:
ans = pd.read_csv(r"C:\Users\hp\桌面\项目\数据挖掘\Data\文本划分\频率统计文本\frequency_A.txt", names=["words", "frequency"])
que = pd.read_csv(r"C:\Users\hp\桌面\项目\数据挖掘\Data\文本划分\频率统计文本\frequency_Q.txt", names=["words", "frequency"])

In [ ]:
que

### 计算tf,idf和tf-idf

In [ ]:
import numpy as np

In [ ]:
total_words = sum(ans['frequency'])
ans['tf'] = ans['frequency'].apply(lambda x: x/total_words)
total_words = sum(que['frequency'])
que['tf'] = que['frequency'].apply(lambda x: x/total_words)
ans['idf'] = ans['words'].apply(lambda x: np.log(2) if x in list(que['words']) else 0)
que['idf'] = que['words'].apply(lambda x: np.log(2) if x in list(ans['words']) else 0)

In [ ]:
ans, que

In [ ]:
ans['tf-idf'] = ans['tf'] * ans['idf']
que['tf-idf'] = que['tf'] * que['idf']

In [ ]:
ans.shape, que.shape

### 删除空值和tf-idf过低的词

In [ ]:
ans.dropna(inplace=True)
que.dropna(inplace=True)

In [ ]:
ans.drop(ans[ans['tf-idf'] < 1e-5].index, inplace=True)
que.drop(que[que['tf-idf'] < 1e-5].index, inplace=True)

In [ ]:
ans.shape, que.shape

In [ ]:
sort_ans = ans.sort_values(by='tf-idf', ascending=False)
sort_que = que.sort_values(by='tf-idf', ascending=False)

In [ ]:
sort_ans

In [ ]:
sort_que

### 将数据保存在csv文件中

In [ ]:
sort_ans.to_csv(r'C:\Users\hp\桌面\项目\数据挖掘\Data\文本划分\tf-idf\answers.csv', index=False)
sort_que.to_csv(r'C:\Users\hp\桌面\项目\数据挖掘\Data\文本划分\tf-idf\questions.csv', index=False)